In [34]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import git
import mlflow
import mlflow.pytorch
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import LSTM, Dense, Dropout
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
from torchsummary import summary

In [35]:
# Указываем MLflow, куда отправлять данные
mlflow.set_tracking_uri("http://213.21.252.250:5000")

# Задаем имя эксперимента
mlflow.set_experiment("LSTM (test)")

# --- Получаем хеш коммита Git ---
try:
    repo = git.Repo(search_parent_directories=True)
    git_commit_hash = repo.head.object.hexsha
except Exception as e:
    git_commit_hash = "N/A" # На случай, если скрипт запущен не из Git-репозитория
    print(f"Warning: Could not get git commit hash. {e}")

print(f"Current Git Commit Hash: {git_commit_hash}")

# --- Параметры, которые нужно логировать ---
# Параметры из скрипта нарезки данных (sample_creator)
data_params = {
    "window_size": 50,
    "step": 1,
    "sampling_rate": 10
}

# Гиперпараметры модели
model_params = {
    "epochs": 5,
    "batch_size": 128,
    "validation_split": 0.2,
    "optimizer": "adam",
    "loss": "mean_squared_error",
    "lr": 0.001
}

Current Git Commit Hash: a616b85e89fd06f5db64b37c1645ef18d8567947


In [36]:
with mlflow.start_run():
    print("Starting MLflow run...")

    # --- Логируем параметры ---
    mlflow.log_params(data_params)
    mlflow.log_params(model_params)
    mlflow.set_tag("git_commit", git_commit_hash)
    print("Parameters logged.")

    def load_and_merge_data(npz_units):
      sample_array_lst = []
      label_array_lst = []
      for npz_unit in npz_units:
        loaded = np.load(npz_unit)
        sample_array_lst.append(loaded['sample'])
        label_array_lst.append(loaded['label'])
      sample_array = np.dstack(sample_array_lst)
      label_array = np.concatenate(label_array_lst)
      sample_array = sample_array.transpose(2, 0, 1)
      return sample_array, label_array

    processed_dir = '../data/processed/'

    # Собираем пути к файлам для train и test
    train_files = [os.path.join(processed_dir, f) for f in os.listdir(processed_dir) if f.startswith(('Unit2_', 'Unit5_', 'Unit10_', 'Unit16_', 'Unit18_', 'Unit20_'))]
    test_files = [os.path.join(processed_dir, f) for f in os.listdir(processed_dir) if f.startswith(('Unit11_', 'Unit14_', 'Unit15_'))]
    print(train_files)

    # Загружаем данные
    X_train, y_train = load_and_merge_data(train_files)
    X_test, y_test = load_and_merge_data(test_files)

    print('Размер обучающей выборки (X):', X_train.shape)
    print('Размер обучающей выборки (y):', y_train.shape)
    print('Размер тестовой выборки (X):', X_test.shape)
    print('Размер тестовой выборки (y):', y_test.shape)

    # Определяем форму входных данных из X_train
    n_timesteps, n_features = X_train.shape[1], X_train.shape[2]

    # --- Определяем модель LSTM на PyTorch ---
    class LSTMModel(nn.Module):
        def __init__(self, input_dim, hidden_dim_1, hidden_dim_2, output_dim=1, dropout_prob=0.2):
            super(LSTMModel, self).__init__()
            # Первый LSTM слой
            self.lstm1 = nn.LSTM(input_dim, hidden_dim_1, batch_first=True)
            # batch_first=True очень важен, чтобы входные данные имели формат (batch, seq, feature), как в Keras
            
            self.dropout1 = nn.Dropout(dropout_prob)
            
            # Второй LSTM слой
            # Он принимает на вход скрытое состояние первого слоя (hidden_dim_1)
            self.lstm2 = nn.LSTM(hidden_dim_1, hidden_dim_2, batch_first=True)
            
            self.dropout2 = nn.Dropout(dropout_prob)
            
            # Полносвязный слой для финального прогноза
            self.fc = nn.Linear(hidden_dim_2, output_dim)

        def forward(self, x):
            # Первый LSTM слой
            # LSTM возвращает output и кортеж (hidden_state, cell_state)
            # Нам нужен output для следующего слоя
            lstm1_out, _ = self.lstm1(x)
            
            # Dropout
            out = self.dropout1(lstm1_out)
            
            # Второй LSTM слой
            # Нам нужен только выход последнего временного шага
            lstm2_out, _ = self.lstm2(out)
            last_hidden_state = lstm2_out[:, -1, :] # Берем выход последнего элемента последовательности
            
            # Dropout
            out = self.dropout2(last_hidden_state)
            
            # Полносвязный слой
            final_output = self.fc(out)
            return final_output

    # --- Подготовка данных для PyTorch ---
    # 1. Преобразуем numpy массивы в torch тензоры
    X_train_tensor = torch.from_numpy(X_train).float()
    y_train_tensor = torch.from_numpy(y_train).float().view(-1, 1) # Убедимся, что форма (batch_size, 1)
    X_test_tensor = torch.from_numpy(X_test).float()
    y_test_tensor = torch.from_numpy(y_test).float().view(-1, 1)

    # 2. Создаем датасеты
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

    # 3. Разделяем на обучающую и валидационную выборки вручную
    val_split = model_params['validation_split']
    dataset_size = len(train_dataset)
    val_size = int(val_split * dataset_size)
    train_size = dataset_size - val_size
    train_subset, val_subset = torch.utils.data.random_split(train_dataset, [train_size, val_size])

    # 4. Создаем загрузчики данных (DataLoader), которые будут подавать данные батчами
    train_loader = DataLoader(dataset=train_subset, batch_size=model_params['batch_size'], shuffle=True)
    val_loader = DataLoader(dataset=val_subset, batch_size=model_params['batch_size'])
    test_loader = DataLoader(dataset=test_dataset, batch_size=model_params['batch_size'])

    # --- Инициализация модели, оптимизатора и функции потерь ---
    model = LSTMModel(input_dim=n_features, hidden_dim_1=16, hidden_dim_2=8)

    # Выбираем функцию потерь (аналог 'loss' в Keras)
    if model_params['loss'] == 'mse':
        criterion = nn.MSELoss()
    else:
        # можно добавить другие функции потерь
        criterion = nn.MSELoss() 

    # Выбираем оптимизатор (аналог 'optimizer' в Keras)
    if model_params['optimizer'] == 'adam':
        optimizer = torch.optim.Adam(model.parameters(), lr=model_params['lr'])
    else:
        # можно добавить другие оптимизаторы
        optimizer = torch.optim.Adam(model.parameters())

    # --- Цикл обучения ---
    for epoch in range(model_params['epochs']):
        model.train() # Переводим модель в режим обучения
        running_loss = 0.0
        for i, (inputs, labels) in enumerate(train_loader):
            # 1. Обнуляем градиенты
            optimizer.zero_grad()
            # 2. Прямой проход (forward pass)
            outputs = model(inputs)
            # 3. Расчет функции потерь
            loss = criterion(outputs, labels)
            # 4. Обратный проход (backward pass)
            loss.backward()
            # 5. Шаг оптимизатора
            optimizer.step()
            running_loss += loss.item()
        
        # Расчет и логирование средней потери за эпоху
        avg_train_loss = running_loss / len(train_loader)
        mlflow.log_metric("train_loss", avg_train_loss, step=epoch)
        # --- Валидация на каждой эпохе --
        model.eval() # Переводим модель в режим оценки
        val_loss = 0.0
        val_mae = 0.0
        with torch.no_grad(): # Отключаем расчет градиентов
            for inputs, labels in val_loader:
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                # Считаем MAE вручну
                val_mae += torch.abs(outputs - labels).sum().item()
        avg_val_loss = val_loss / len(val_loader)
        avg_val_mae = val_mae / len(val_subset)
        mlflow.log_metric("val_loss", avg_val_loss, step=epoch)
        mlflow.log_metric("val_mae", avg_val_mae, step=epoch)
        print(f"Epoch [{epoch+1}/{model_params['epochs']}], rain oss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}, Val MAE: {avg_val_mae:.4f}")
        # --- Оценка на тестовых данных ---
        model.eval()
        test_mae = 0.0
        with torch.no_grad():
            for inputs, labels in test_loader:
                outputs = model(inputs)
                test_mae += torch.abs(outputs - labels).sum().item()
        
        final_mae = test_mae / len(test_dataset)
        print(f'\nTest MAE: {final_mae:.2f}')
        # Логирование финальных метрик
        metrics = {"mae": final_mae}
        mlflow.log_metrics(metrics)
        print(f"Metrics logged: {metrics}")
        
        # --- Логирование самой модели ---
        mlflow.pytorch.log_model(
            model,
            artifact_path="lstm-model"
        )

    print("Training finished.")

    # # --- Создаем простую LSTM модель ---
    # model = Sequential()
    # model.add(LSTM(16, input_shape=(n_timesteps, n_features), return_sequences=True)) # return_sequences=True, если следующий слой тоже LSTM
    # model.add(Dropout(0.2))
    # model.add(LSTM(8))
    # model.add(Dropout(0.2))
    # model.add(Dense(1)) # Один выход, так как предсказываем одно число - RUL

    # # Компилируем модель
    # model.compile(optimizer=model_params['optimizer'], loss=model_params['loss'], metrics=['mae'])

    # summary_list = []
    # model.summary(print_fn=lambda x: summary_list.append(x))
    # model_summary_string = "\n".join(summary_list)

    # mlflow.log_text(model_summary_string, 'model_summary.txt')

    # model.summary()

    # # --- Обучаем модель ---
    # history = model.fit(X_train, y_train, 
    #                     epochs=model_params['epochs'], 
    #                     batch_size=model_params['batch_size'], 
    #                     validation_split=model_params['validation_split'], # Используем часть данных для валидации на лету
    #                     callbacks=[mlflow.keras.MLflowCallback()],
    #                     verbose=1)

    # # --- Оцениваем на тестовых данных ---
    # loss, mae = model.evaluate(X_test, y_test, verbose=0)
    # print(f'\nTest MAE: {mae:.2f}')

    # metrics = {
    #     "mae": mae
    # }
    # mlflow.log_metrics(metrics)
    # print(f"Metrics logged: {metrics}")

    # # --- Логирование самой модели ---
    # mlflow.keras.log_model(
    #     model,
    #     artifact_path="lstm-model", # Название папки с моделью в MLflow
    # )

print("Model logged as an artifact.")

print("MLflow run finished successfully!")

Starting MLflow run...
Parameters logged.
['../data/processed/Unit16_win50_str1_smp10.npz', '../data/processed/Unit5_win50_str1_smp10.npz', '../data/processed/Unit18_win50_str1_smp10.npz', '../data/processed/Unit20_win50_str1_smp10.npz', '../data/processed/Unit2_win50_str1_smp10.npz', '../data/processed/Unit10_win50_str1_smp10.npz']
🏃 View run glamorous-steed-925 at: http://213.21.252.250:5000/#/experiments/1/runs/49647fe4ac4645a9871db8d7bf6781ab
🧪 View experiment at: http://213.21.252.250:5000/#/experiments/1


KeyboardInterrupt: 